In [1]:
import pandas as pd
pd.set_option('display.max_columns', None) 
import numpy as np
import pickle

import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
input_path = 'H:\RediMinds\VCQI'
df_filter2_codes = pd.read_csv(input_path+"\VCQI_clean.csv")

In [3]:
# sepearte target variable from predictors for furthur processing
X = df_filter2_codes.drop(columns=['INTRA_OP_COMPLICATIONS'])
Y = df_filter2_codes['INTRA_OP_COMPLICATIONS']
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3312 entries, 0 to 3311
Data columns (total 33 columns):
PROC NAME                          3312 non-null int64
GENDER                             3312 non-null int64
AGE AT SURGERY                     3312 non-null float64
MARITAL STATUS                     3312 non-null int64
RACE                               3312 non-null int64
WEIGHT                             3312 non-null float64
HEIGHT                             3312 non-null float64
BMI                                3312 non-null float64
CLINICAL SIZE (mm)                 3312 non-null float64
SYMPTOMS                           3312 non-null int64
SOLITARY KIDNEY                    3312 non-null int64
BILATERALITY OF TUMOR              3312 non-null int64
SIDE OF TUMOR                      3312 non-null int64
SIDE OF SURGERY                    3312 non-null int64
FACE                               3312 non-null int64
TUMOR lOCATION                     3312 non-null int64
PRE

In [4]:
X.select_dtypes('float').columns

Index(['AGE AT SURGERY', 'WEIGHT', 'HEIGHT', 'BMI', 'CLINICAL SIZE (mm)',
       'PRE-OP HB', 'PRE-OP HT', 'PRE-OP WBC', 'PRE-OP CREAT', 'PRE-OP EGFR',
       'NO OF LESIONS'],
      dtype='object')

In [7]:
# Create dummy variables
with open (input_path+'\cat_col', 'rb') as fp:
    cat_col = pickle.load(fp)


nn_data = X.copy()
for col in cat_col:
    if not col == 'INTRA_OP_COMPLICATIONS':
        nn_data = nn_data.merge(pd.get_dummies(X[col], prefix=col), how = 'left', left_index = True, right_index =True) # create dummy variables
        nn_data = nn_data.drop(labels = col, axis = 'columns') # drop original variables

In [8]:
nn_data.head()

,AGE AT SURGERY,WEIGHT,HEIGHT,BMI,CLINICAL SIZE (mm),PRE-OP HB,PRE-OP HT,PRE-OP WBC,PRE-OP CREAT,PRE-OP EGFR,NO OF LESIONS,PROC NAME_0,GENDER_0,GENDER_1,GENDER_2,MARITAL STATUS_0,MARITAL STATUS_1,MARITAL STATUS_2,MARITAL STATUS_3,MARITAL STATUS_4,RACE_0,RACE_1,RACE_2,RACE_3,RACE_4,SYMPTOMS_0,SYMPTOMS_1,SYMPTOMS_2,SYMPTOMS_3,SOLITARY KIDNEY_0,SOLITARY KIDNEY_1,SOLITARY KIDNEY_2,BILATERALITY OF TUMOR_0,BILATERALITY OF TUMOR_1,BILATERALITY OF TUMOR_2,SIDE OF TUMOR_0,SIDE OF TUMOR_1,SIDE OF TUMOR_2,SIDE OF TUMOR_3,SIDE OF SURGERY_0,SIDE OF SURGERY_1,SIDE OF SURGERY_2,SIDE OF SURGERY_3,FACE_0,FACE_1,FACE_2,TUMOR lOCATION_0,TUMOR lOCATION_1,TUMOR lOCATION_2,TUMOR lOCATION_3,POLAR LOCATION_0,POLAR LOCATION_1,POLAR LOCATION_2,RENAL SINUS_0,RENAL SINUS_1,RENAL SINUS_2,RENAL SINUS_3,RENAL SINUS_4,EXOPHYTIC RATE_0,EXOPHYTIC RATE_1,EXOPHYTIC RATE_2,EXOPHYTIC RATE_3,CLINICAL SIZE GROUP_0,CLINICAL SIZE GROUP_1,CLINICAL SIZE GROUP_2,CLINICAL SIZE GROUP_3,CT_0,CT_1,CT_2,CT_3,CT_4,CT_5,CT_6,CN_0,CN_1,CN_2,CN_3,CM_0,CM_1,CM_2,CM_3,RADIUS (maximal diameter in cm)_0,RADIUS (maximal diameter in cm)_1,RADIUS (maximal diameter in cm)_2,RADIUS (maximal diameter in cm)_3,EXOPHYTIC/ENDOPHYTIC PROPERTIES_0,EXOPHYTIC/ENDOPHYTIC PROPERTIES_1,EXOPHYTIC/ENDOPHYTIC PROPERTIES_2,EXOPHYTIC/ENDOPHYTIC PROPERTIES_3,ANTERIOR OR POSTERIOR_0,ANTERIOR OR POSTERIOR_1,ANTERIOR OR POSTERIOR_2,ANTERIOR OR POSTERIOR_3,PARTIAL NEPHRO INDICATION_0,PARTIAL NEPHRO INDICATION_1,PARTIAL NEPHRO INDICATION_2,PARTIAL NEPHRO INDICATION_3
0,50.0,57.00,168.0,20.20,7.0,13.2,50.0,7800.0,0.63,100.030000,1.0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0
1,69.0,68.60,161.3,26.37,74.0,14.7,44.2,6400.0,0.80,71.119672,1.0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1
2,83.0,83.65,164.0,31.10,80.0,12.5,38.9,5590.0,0.97,54.844682,1.0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0
3,66.0,107.00,180.0,32.78,73.0,17.0,50.8,8000.0,0.93,81.290783,1.0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0
4,61.0,82.90,171.0,28.35,8.0,12.1,37.1,7710.0,1.10,68.053260,1.0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0


In [9]:
# Split data in test and train
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(nn_data, Y, test_size=0.30, random_state=42, stratify = Y)

from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(nn_data, Y, test_size=0.20, random_state=42, stratify = Y)

Using TensorFlow backend.


In [10]:
print("Number of records in trainset {}".format(len(x_train)))
print("Number records in testset {}".format(len(x_test)))

Number of records in trainset 2649
Number records in testset 994


In [11]:
sum(y_train)/len(y_train)

0.04983012457531144

In [12]:
sum(y_test)/len(y_test)

0.05030181086519115

In [13]:
x_train.describe()

,AGE AT SURGERY,WEIGHT,HEIGHT,BMI,CLINICAL SIZE (mm),PRE-OP HB,PRE-OP HT,PRE-OP WBC,PRE-OP CREAT,PRE-OP EGFR,NO OF LESIONS,PROC NAME_0,GENDER_0,GENDER_1,GENDER_2,MARITAL STATUS_0,MARITAL STATUS_1,MARITAL STATUS_2,MARITAL STATUS_3,MARITAL STATUS_4,RACE_0,RACE_1,RACE_2,RACE_3,RACE_4,SYMPTOMS_0,SYMPTOMS_1,SYMPTOMS_2,SYMPTOMS_3,SOLITARY KIDNEY_0,SOLITARY KIDNEY_1,SOLITARY KIDNEY_2,BILATERALITY OF TUMOR_0,BILATERALITY OF TUMOR_1,BILATERALITY OF TUMOR_2,SIDE OF TUMOR_0,SIDE OF TUMOR_1,SIDE OF TUMOR_2,SIDE OF TUMOR_3,SIDE OF SURGERY_0,SIDE OF SURGERY_1,SIDE OF SURGERY_2,SIDE OF SURGERY_3,FACE_0,FACE_1,FACE_2,TUMOR lOCATION_0,TUMOR lOCATION_1,TUMOR lOCATION_2,TUMOR lOCATION_3,POLAR LOCATION_0,POLAR LOCATION_1,POLAR LOCATION_2,RENAL SINUS_0,RENAL SINUS_1,RENAL SINUS_2,RENAL SINUS_3,RENAL SINUS_4,EXOPHYTIC RATE_0,EXOPHYTIC RATE_1,EXOPHYTIC RATE_2,EXOPHYTIC RATE_3,CLINICAL SIZE GROUP_0,CLINICAL SIZE GROUP_1,CLINICAL SIZE GROUP_2,CLINICAL SIZE GROUP_3,CT_0,CT_1,CT_2,CT_3,CT_4,CT_5,CT_6,CN_0,CN_1,CN_2,CN_3,CM_0,CM_1,CM_2,CM_3,RADIUS (maximal diameter in cm)_0,RADIUS (maximal diameter in cm)_1,RADIUS (maximal diameter in cm)_2,RADIUS (maximal diameter in cm)_3,EXOPHYTIC/ENDOPHYTIC PROPERTIES_0,EXOPHYTIC/ENDOPHYTIC PROPERTIES_1,EXOPHYTIC/ENDOPHYTIC PROPERTIES_2,EXOPHYTIC/ENDOPHYTIC PROPERTIES_3,ANTERIOR OR POSTERIOR_0,ANTERIOR OR POSTERIOR_1,ANTERIOR OR POSTERIOR_2,ANTERIOR OR POSTERIOR_3,PARTIAL NEPHRO INDICATION_0,PARTIAL NEPHRO INDICATION_1,PARTIAL NEPHRO INDICATION_2,PARTIAL NEPHRO INDICATION_3
count,2649.000000,2649.00000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.0,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.00000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000,2649.000000
mean,57.811808,81.66292,169.426874,28.720060,32.920257,13.777384,41.105096,7636.610072,0.977267,80.278006,1.055568,1.0,0.352586,0.644017,0.003398,0.021518,0.518686,0.368441,0.071348,0.020008,0.026048,0.083428,0.388826,0.127595,0.374103,0.580974,0.152888,0.251416,0.014723,0.157795,0.814647,0.027558,0.393733,0.576444,0.029823,0.022273,0.318988,0.320876,0.337863,0.000755,0.491884,0.003775,0.503586,0.401661,0.246508,0.351831,0.259343,0.316723,0.152888,0.271046,0.223858,0.320121,0.456021,0.380898,0.000378,0.419404,0.078898,0.120423,0.289921,0.352963,0.086448,0.270668,0.215553,0.579086,0.023783,0.181578,0.115893,0.630049,0.220461,0.024538,0.003398,0.005285,0.000378,0.003775,0.537939,0.207626,0.250661,0.014345,0.534919,0.223103,0.227633,0.232163,0.625142,0.03020,0.112495,0.377123,0.351076,0.094753,0.177048,0.329181,0.201208,0.140053,0.329558,0.029068,0.584371,0.283126,0.103435
std,12.590646,15.66466,9.146982,5.466842,16.203767,1.581694,4.185438,2366.978040,0.402954,21.255534,0.249683,0.0,0.477865,0.478901,0.058200,0.145129,0.499745,0.482473,0.257453,0.140052,0.159307,0.276580,0.487576,0.333702,0.483982,0.493493,0.359947,0.433909,0.120463,0.364618,0.388657,0.163732,0.488669,0.494215,0.170130,0.147596,0.466173,0.466901,0.473071,0.027472,0.500029,0.061337,0.500082,0.490327,0.431059,0.477632,0.438357,0.465286,0.359947

In [14]:
# Standardize the numeric columns
from sklearn.preprocessing import StandardScaler
stdc = StandardScaler()
x_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
x_valid.reset_index(drop=True, inplace=True)
y_valid.reset_index(drop=True, inplace=True)
x_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
float_col = X.select_dtypes('float').columns
x_train[float_col] = pd.DataFrame(stdc.fit_transform(x_train[float_col]),columns=float_col).copy()
x_valid[float_col] = pd.DataFrame(stdc.fit_transform(x_valid[float_col]),columns=float_col).copy()
x_test[float_col] = pd.DataFrame(stdc.transform(x_test[float_col]),columns=float_col).copy()

H:\Anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
H:\Anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
H:\Anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

In [15]:
sm = SMOTE(random_state=2)
x_train, y_train = sm.fit_sample(x_train, y_train)

In [16]:
x_train = pd.DataFrame(x_train, columns=x_test.columns).copy()
y_train = pd.Series(y_train)
x_train.head()

,AGE AT SURGERY,WEIGHT,HEIGHT,BMI,CLINICAL SIZE (mm),PRE-OP HB,PRE-OP HT,PRE-OP WBC,PRE-OP CREAT,PRE-OP EGFR,NO OF LESIONS,PROC NAME_0,GENDER_0,GENDER_1,GENDER_2,MARITAL STATUS_0,MARITAL STATUS_1,MARITAL STATUS_2,MARITAL STATUS_3,MARITAL STATUS_4,RACE_0,RACE_1,RACE_2,RACE_3,RACE_4,SYMPTOMS_0,SYMPTOMS_1,SYMPTOMS_2,SYMPTOMS_3,SOLITARY KIDNEY_0,SOLITARY KIDNEY_1,SOLITARY KIDNEY_2,BILATERALITY OF TUMOR_0,BILATERALITY OF TUMOR_1,BILATERALITY OF TUMOR_2,SIDE OF TUMOR_0,SIDE OF TUMOR_1,SIDE OF TUMOR_2,SIDE OF TUMOR_3,SIDE OF SURGERY_0,SIDE OF SURGERY_1,SIDE OF SURGERY_2,SIDE OF SURGERY_3,FACE_0,FACE_1,FACE_2,TUMOR lOCATION_0,TUMOR lOCATION_1,TUMOR lOCATION_2,TUMOR lOCATION_3,POLAR LOCATION_0,POLAR LOCATION_1,POLAR LOCATION_2,RENAL SINUS_0,RENAL SINUS_1,RENAL SINUS_2,RENAL SINUS_3,RENAL SINUS_4,EXOPHYTIC RATE_0,EXOPHYTIC RATE_1,EXOPHYTIC RATE_2,EXOPHYTIC RATE_3,CLINICAL SIZE GROUP_0,CLINICAL SIZE GROUP_1,CLINICAL SIZE GROUP_2,CLINICAL SIZE GROUP_3,CT_0,CT_1,CT_2,CT_3,CT_4,CT_5,CT_6,CN_0,CN_1,CN_2,CN_3,CM_0,CM_1,CM_2,CM_3,RADIUS (maximal diameter in cm)_0,RADIUS (maximal diameter in cm)_1,RADIUS (maximal diameter in cm)_2,RADIUS (maximal diameter in cm)_3,EXOPHYTIC/ENDOPHYTIC PROPERTIES_0,EXOPHYTIC/ENDOPHYTIC PROPERTIES_1,EXOPHYTIC/ENDOPHYTIC PROPERTIES_2,EXOPHYTIC/ENDOPHYTIC PROPERTIES_3,ANTERIOR OR POSTERIOR_0,ANTERIOR OR POSTERIOR_1,ANTERIOR OR POSTERIOR_2,ANTERIOR OR POSTERIOR_3,PARTIAL NEPHRO INDICATION_0,PARTIAL NEPHRO INDICATION_1,PARTIAL NEPHRO INDICATION_2,PARTIAL NEPHRO INDICATION_3
0,-0.223367,-1.319328,-0.484062,-1.156290,-1.414769,-0.554816,0.333338,0.195810,-0.315895,0.010917,-0.222597,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.332706,-1.702428,-0.593409,-1.503905,-1.044415,0.204007,0.452823,0.001779,-0.464823,-0.107193,-0.222597,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-1.335514,-0.011679,-0.008406,-0.680606,-1.353043,-0.004669,-0.013166,0.001779,-0.191788,1.869136,-0.022305,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,-0.143928,-0.011679,-0.008406,0.124399,-0.488883,-0.004669,-0.013166,0.001779,1.545712,-1.518856,-0.022305,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.223367,-0.011679,-0.008406,-0.005500,-0.488883,-0.004669,-0.013166,0.001779,-0.812323,0.928027,-0.222597,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [65]:
# input layer
input_layer = keras.layers.Input(shape=(x_train.shape[1],), name = "input_layer")
x = keras.layers.Dense(128, name = 'Dense_1',activation='relu')(input_layer)
x = keras.layers.Dropout(0.4, name=  'Dropout_1')(x)
x = keras.layers.Dense(128, name = 'Dense_2',activation='relu')(x)
x = keras.layers.Dropout(0.4, name=  'Dropout_2')(x)
main_output = keras.layers.Dense(1, activation='sigmoid',name='main_output')(x)

model = keras.Model(inputs= input_layer, outputs=main_output)

# compiling the model
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC(curve = 'ROC',name = 'AUC_ROC'),
                       tf.keras.metrics.AUC(curve = 'PR', name = 'AUC_PR')],
              )

# Keras callback. The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min',restore_best_weights=True)


model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 97)]              0         
_________________________________________________________________
Dense_1 (Dense)              (None, 128)               12544     
_________________________________________________________________
Dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
Dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
Dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
main_output (Dense)          (None, 1)                 129       
Total params: 29,185
Trainable params: 29,185
Non-trainable params: 0
_______________________________________________________

In [66]:
model.fit(x = x_train, y = y_train,
          validation_data = (x_valid, y_valid),
          epochs = 50,
          callbacks = [early_stop]
     )


Train on 5034 samples, validate on 663 samples
Epoch 1/50
5034/5034 [==============================] - ETA: 29s - loss: 0.7394 - AUC_ROC: 0.4941 - AUC_PR: 0.484 - ETA: 1s - loss: 0.7460 - AUC_ROC: 0.4800 - AUC_PR: 0.479 - ETA: 0s - loss: 0.7363 - AUC_ROC: 0.4972 - AUC_PR: 0.50 - ETA: 0s - loss: 0.7255 - AUC_ROC: 0.5172 - AUC_PR: 0.50 - ETA: 0s - loss: 0.7178 - AUC_ROC: 0.5326 - AUC_PR: 0.50 - ETA: 0s - loss: 0.7098 - AUC_ROC: 0.5513 - AUC_PR: 0.52 - ETA: 0s - loss: 0.7011 - AUC_ROC: 0.5707 - AUC_PR: 0.53 - ETA: 0s - loss: 0.6926 - AUC_ROC: 0.5895 - AUC_PR: 0.55 - 1s 264us/sample - loss: 0.6894 - AUC_ROC: 0.5960 - AUC_PR: 0.5672 - val_loss: 0.6191 - val_AUC_ROC: 0.7866 - val_AUC_PR: 0.1517
Epoch 2/50
5034/5034 [==============================] - ETA: 0s - loss: 0.5892 - AUC_ROC: 0.7520 - AUC_PR: 0.79 - ETA: 0s - loss: 0.6310 - AUC_ROC: 0.7094 - AUC_PR: 0.68 - ETA: 0s - loss: 0.6156 - AUC_ROC: 0.7374 - AUC_PR: 0.71 - ETA: 0s - loss: 0.6096 - AUC_ROC: 0.7468 - AUC_PR: 0.71 - ETA: 0s - loss

Epoch 14/50
5034/5034 [==============================] - ETA: 0s - loss: 0.3556 - AUC_ROC: 0.9048 - AUC_PR: 0.89 - ETA: 0s - loss: 0.3077 - AUC_ROC: 0.9413 - AUC_PR: 0.93 - ETA: 0s - loss: 0.3035 - AUC_ROC: 0.9443 - AUC_PR: 0.93 - ETA: 0s - loss: 0.3057 - AUC_ROC: 0.9429 - AUC_PR: 0.93 - ETA: 0s - loss: 0.3086 - AUC_ROC: 0.9407 - AUC_PR: 0.93 - ETA: 0s - loss: 0.3057 - AUC_ROC: 0.9417 - AUC_PR: 0.92 - ETA: 0s - loss: 0.3064 - AUC_ROC: 0.9415 - AUC_PR: 0.92 - 0s 75us/sample - loss: 0.3026 - AUC_ROC: 0.9433 - AUC_PR: 0.9297 - val_loss: 0.3020 - val_AUC_ROC: 0.8303 - val_AUC_PR: 0.2841
Epoch 15/50
5034/5034 [==============================] - ETA: 0s - loss: 0.3292 - AUC_ROC: 0.9176 - AUC_PR: 0.91 - ETA: 0s - loss: 0.3101 - AUC_ROC: 0.9376 - AUC_PR: 0.92 - ETA: 0s - loss: 0.2967 - AUC_ROC: 0.9452 - AUC_PR: 0.92 - ETA: 0s - loss: 0.2899 - AUC_ROC: 0.9493 - AUC_PR: 0.93 - ETA: 0s - loss: 0.2875 - AUC_ROC: 0.9504 - AUC_PR: 0.93 - ETA: 0s - loss: 0.2878 - AUC_ROC: 0.9500 - AUC_PR: 0.93 - ETA: 

5034/5034 [==============================] - ETA: 0s - loss: 0.1647 - AUC_ROC: 0.9957 - AUC_PR: 0.99 - ETA: 0s - loss: 0.1714 - AUC_ROC: 0.9821 - AUC_PR: 0.97 - ETA: 0s - loss: 0.1809 - AUC_ROC: 0.9801 - AUC_PR: 0.97 - ETA: 0s - loss: 0.1840 - AUC_ROC: 0.9773 - AUC_PR: 0.97 - ETA: 0s - loss: 0.1792 - AUC_ROC: 0.9792 - AUC_PR: 0.97 - ETA: 0s - loss: 0.1813 - AUC_ROC: 0.9784 - AUC_PR: 0.97 - ETA: 0s - loss: 0.1797 - AUC_ROC: 0.9790 - AUC_PR: 0.97 - 0s 76us/sample - loss: 0.1766 - AUC_ROC: 0.9797 - AUC_PR: 0.9739 - val_loss: 0.2442 - val_AUC_ROC: 0.7989 - val_AUC_PR: 0.2856
Epoch 28/50
5034/5034 [==============================] - ETA: 0s - loss: 0.2776 - AUC_ROC: 0.9490 - AUC_PR: 0.93 - ETA: 0s - loss: 0.1694 - AUC_ROC: 0.9822 - AUC_PR: 0.97 - ETA: 0s - loss: 0.1708 - AUC_ROC: 0.9818 - AUC_PR: 0.97 - ETA: 0s - loss: 0.1704 - AUC_ROC: 0.9813 - AUC_PR: 0.97 - ETA: 0s - loss: 0.1673 - AUC_ROC: 0.9820 - AUC_PR: 0.97 - ETA: 0s - loss: 0.1671 - AUC_ROC: 0.9822 - AUC_PR: 0.97 - ETA: 0s - loss: 0

Epoch 41/50
5034/5034 [==============================] - ETA: 0s - loss: 0.0646 - AUC_ROC: 1.0000 - AUC_PR: 1.00 - ETA: 0s - loss: 0.1031 - AUC_ROC: 0.9927 - AUC_PR: 0.99 - ETA: 0s - loss: 0.1045 - AUC_ROC: 0.9930 - AUC_PR: 0.99 - ETA: 0s - loss: 0.0932 - AUC_ROC: 0.9948 - AUC_PR: 0.99 - ETA: 0s - loss: 0.1001 - AUC_ROC: 0.9930 - AUC_PR: 0.99 - ETA: 0s - loss: 0.1069 - AUC_ROC: 0.9913 - AUC_PR: 0.98 - ETA: 0s - loss: 0.1052 - AUC_ROC: 0.9917 - AUC_PR: 0.98 - 0s 74us/sample - loss: 0.1067 - AUC_ROC: 0.9912 - AUC_PR: 0.9878 - val_loss: 0.2559 - val_AUC_ROC: 0.7709 - val_AUC_PR: 0.2935
Epoch 42/50
5034/5034 [==============================] - ETA: 0s - loss: 0.0781 - AUC_ROC: 0.9961 - AUC_PR: 0.99 - ETA: 0s - loss: 0.0791 - AUC_ROC: 0.9978 - AUC_PR: 0.99 - ETA: 0s - loss: 0.1012 - AUC_ROC: 0.9936 - AUC_PR: 0.99 - ETA: 0s - loss: 0.0981 - AUC_ROC: 0.9939 - AUC_PR: 0.99 - ETA: 0s - loss: 0.1000 - AUC_ROC: 0.9933 - AUC_PR: 0.99 - ETA: 0s - loss: 0.0985 - AUC_ROC: 0.9937 - AUC_PR: 0.99 - ETA: 

In [67]:
# predict on test set
pd.DataFrame(model.predict(x_test))

,0
0,2.463580e-01
1,3.700644e-03
2,1.853615e-03
3,2.212405e-03
4,2.657473e-04
5,9.954870e-04
6,9.968877e-05
7,1.381063e-01
8,2.846718e-04
9,1.938641e-04


In [68]:
results_nn = pd.DataFrame(model.predict(x_test), columns=['pred_prob'])
results_nn['pred_label'] =  results_nn['pred_prob'].apply(lambda x: 1 if x>=0.5 else 0)
results_nn['true_label'] = np.array(y_test)

In [69]:
# NeuralNetwork Score Raw Data
print("\n Model Balanced Accuracy: \n" + str(metrics.balanced_accuracy_score(results_nn['true_label'], results_nn['pred_label'])))
print("\n Confusion Matrix : \n"+str(metrics.confusion_matrix(results_nn['true_label'], results_nn['pred_label'])))
print("\n Classification Report: \n"+ str(metrics.classification_report(results_nn['true_label'], results_nn['pred_label'])))
print("\n AUC-ROC: \n"+ str(metrics.roc_auc_score(results_nn['true_label'], results_nn['pred_prob'])))


def calc_aucpr_data(result):
    y_ACTUAL = result['true_label']
    scores_prob = result['pred_prob']
    yhat = result['pred_label']
    precision, recall, thresholds = metrics.precision_recall_curve(y_ACTUAL, scores_prob, pos_label=1)
    prc_auc = metrics.auc(recall,precision)
    return prc_auc

print("\n AUC-ROC: \n"+ str(calc_aucpr_data(results_nn)))


 Model Balanced Accuracy: 
0.7741101694915253

 Confusion Matrix : 
[[914  30]
 [ 21  29]]

 Classification Report: 
              precision    recall  f1-score   support

           0       0.98      0.97      0.97       944
           1       0.49      0.58      0.53        50

    accuracy                           0.95       994
   macro avg       0.73      0.77      0.75       994
weighted avg       0.95      0.95      0.95       994


 AUC-ROC: 
0.8664194915254237

 AUC-ROC: 
0.45656147664885705


In [70]:
def bootstrapped_AUC(result):
    from sklearn.utils import resample
    from tqdm import tqdm

    n_iter = 10000
    roc_auc = list()
    prc_auc = list()


    for i in range(n_iter):
        result_sample = resample(result, n_samples = len(result),random_state=i)
        
        #Calculating AUROC for each sample
        y_ACTUAL= result_sample['true_label']
        scores_prob = result_sample['pred_prob']
        fpr, tpr, thresholds = metrics.roc_curve(y_ACTUAL, scores_prob, pos_label=1)
        roc_auc.append(metrics.auc(fpr, tpr))

        #calculate AUPRC for each sample
        y_ACTUAL = result_sample['true_label']
        scores_prob = result_sample['pred_prob']
        yhat = result_sample['pred_label']
        precision, recall, thresholds = metrics.precision_recall_curve(y_ACTUAL, scores_prob, pos_label=1)
        prc_auc.append(metrics.auc(recall,precision))
    
    return roc_auc, prc_auc

In [71]:
roc_auc_nn, pr_auc_nn = bootstrapped_AUC(results_nn)

In [72]:
dict = {'roc_auc_nn': roc_auc_nn,
        'pr_auc_nn': pr_auc_nn,
       }
pd.DataFrame(dict).describe(percentiles=[0.025,0.975])

,roc_auc_nn,pr_auc_nn
count,10000.000000,10000.000000
mean,0.865948,0.457294
std,0.029427,0.076309
min,0.726604,0.187654
2.5%,0.804240,0.305795
50%,0.867475,0.458310
97.5%,0.920592,0.604137
max,0.961045,0.754621


In [75]:
# Save the model
model.save('nn_smote.h5')